In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
from matplotlib import pyplot as plt
from pyscf import scf, gto, lib, grad, hessian, dft
from functools import partial

import sys, gc, os
sys.path.append('../../')
os.environ["LOGLEVEL"] = "2"
import pickle
from utilities import val_from_fchk, timing
from hf_helper import HFHelper
from numeric_helper import NumericDiff
from gga_helper import GGAHelper
from grid_helper import GridHelper, KernelHelper

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 10 / 8])
np.einsum_path = partial(np.einsum_path, optimize=["greedy", 1024 ** 3 * 10 / 8])
np.set_printoptions(5, linewidth=120, suppress=True)

In [2]:
# Libxc does not provide B3LYP 3th order of derivative, move to xcfun
dft.numint.libxc = dft.xcfun

In [3]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! NOTE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For clear and simplicity, following code cell will be hidden and assumed to be executed!

mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

grids = dft.gen_grid.Grids(mol)
grids.atom_grid = (75, 302)
grids.becke_scheme = dft.gen_grid.stratmann
grids.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

In [ ]:
scfh = GGAHelper(mol, "b3lypg", grids)
scfh.get_kerh(deriv=3)

scfh.get_grad()
scfh.get_hess()

scf_eng  = scfh.scf_eng
scf_grad = scfh.scf_grad
scf_hess = scfh.scf_hess

In [ ]:
cx      = scfh.cx
C       = scfh.C
Co      = scfh.Co
Cv      = scfh.Cv
e       = scfh.e
eo      = scfh.eo
ev      = scfh.ev
D       = scfh.D
F_0_ao  = scfh.F_0_ao
F_0_mo  = scfh.F_0_mo
H_0_ao  = scfh.H_0_ao
H_0_mo  = scfh.H_0_mo
eri0_ao = scfh.eri0_ao
eri0_mo = scfh.eri0_mo
mo_occ  = scfh.mo_occ
H_1_ao  = scfh.H_1_ao
H_1_mo  = scfh.H_1_mo
S_1_ao  = scfh.S_1_ao
S_1_mo  = scfh.S_1_mo
F_1_ao  = scfh.F_1_ao
F_1_mo  = scfh.F_1_mo
eri1_ao = scfh.eri1_ao
eri1_mo = scfh.eri1_mo
H_2_ao  = scfh.H_2_ao
H_2_mo  = scfh.H_2_mo
S_2_ao  = scfh.S_2_ao
S_2_mo  = scfh.S_2_mo
eri2_ao = scfh.eri2_ao
B_1     = scfh.B_1
U_1     = scfh.U_1
Xi_2    = scfh.Xi_2
U_2_vo  = scfh.U_2_vo

Ax0_Core = scfh.Ax0_Core
# Ax1_Core = scfh.Ax1_Core
mol_slice = scfh.mol_slice

In [ ]:
grdh = GridHelper(mol, grids, D)
kerh = KernelHelper(grdh, scfh.xc, deriv=3)

## First order U revisit

In [ ]:
# %%time
# C_diff = NumericDiff(mol, lambda mol : GGAHelper(mol, "b3lypg", grids).C, interval=1e-4).get_numdif()
# U_1_num = np.linalg.inv(C) @ C_diff

In [ ]:
# with open('num_C_U1.dat', 'wb') as f:
#     pickle.dump({"C_diff": C_diff, "U_1_num": U_1_num}, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('num_C_U1.dat', 'rb') as f:
    pickle_load = pickle.load(f)
C_diff = pickle_load["C_diff"]
U_1_num = pickle_load["U_1_num"]

In [ ]:
plt.hist(abs(U_1_num - U_1).ravel(), bins=np.logspace(np.log10(1e-8),np.log10(1e1), 50))
plt.gca().set_xscale("log")

In [ ]:
plt.hist(abs(U_1_num).ravel(), bins=np.logspace(np.log10(1e-8),np.log10(1e1), 50))
plt.gca().set_xscale("log")

In [ ]:
with open('num_U1_U2.dat', 'rb') as f:
    pickle_load = pickle.load(f)
U1vo_diff = pickle_load["U_1_vo_diff"]
U2vo_num = pickle_load["U_2_vo_num"]

In [ ]:
plt.hist(abs(U2vo_num - U_2_vo).ravel(), bins=np.logspace(np.log10(1e-8),np.log10(1e1), 50))
plt.gca().set_xscale("log")

In [ ]:
plt.hist(abs(U2vo_num).ravel(), bins=np.logspace(np.log10(1e-8),np.log10(1e1), 50))
plt.gca().set_xscale("log")

In [ ]:
conv = (
    + U2vo_num * lib.direct_sum("a - i", ev, eo)
    + scfh.Ax0_Core(sv, so, sv, so)(U2vo_num)
    + scfh.B_2_vo
)

In [ ]:
print(conv.max())
print(conv.min())